In [1]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import torch
import os
import sys
import glob
import pickle
import copy
import random
import time
import traceback
import shutil

from multiprocessing import Process, Manager

# Config

In [2]:
PROJECT_ROOT = '/workspace/GREAT_ASV_system/'
sys.path.append(PROJECT_ROOT)

In [3]:
# OPT_INDEX = '/workspace/DATASET/server9/STD_SRE_EGS'
# if not os.path.exists(OPT_INDEX):
#     os.mkdir(OPT_INDEX)
    
OPT_INDEX = '/workspace/DATASET/server9_ssd/STD_SRE_EGS'
if not os.path.exists(OPT_INDEX):
    os.mkdir(OPT_INDEX)

In [8]:
SRE18_TEST_DIR = '/workspace/DATASET/server9/SRE18_TEST'
AUG_INFO_DIR = '/workspace/DATASET/server9/STD_musan&rir_info'

In [9]:
INFO_DIR = os.path.join(OPT_INDEX, 'SRE18_INDO_INFO')
if not os.path.exists(INFO_DIR):
    os.mkdir(INFO_DIR)

# spk2id = os.path.join(INFO_DIR, 'spk2id')
# spk2utt_dict = os.path.join(INFO_DIR, 'spk2utt_dict')
spk2utt_train_dict = os.path.join(INFO_DIR, 'spk2utt_train_dict')
# spk2utt_val_dict = os.path.join(INFO_DIR, 'spk2utt_val_dict')
spk2utt_train_len = os.path.join(INFO_DIR, 'spk2utt_train_len')
# spk2utt_val_len = os.path.join(INFO_DIR, 'spk2utt_val_len')

In [10]:
# val_utt_num = 200

# Make spk2utt dict

In [10]:
SRE16_TEST_DIR+'/Unlabeled_WeakS'

'/workspace/DATASET/server9/SRE18_TEST/Unlabeled_WeakS'

In [12]:
# spk2utt = {}
spk2utt_train = {}
# spk2utt_val = {}

# with open(spk2id, 'w') as f:
#     pass

label_dir = glob.glob(SRE18_TEST_DIR+'/Unlabeled_WeakS/*')
# label = [i.split('/')[-1] for i in label_dir]

for count, i in enumerate(label_dir):
#     with open(spk2id, 'a') as f:
#         f.write(str(count)+','+i+'\n')
    spk2utt_train[count] = glob.glob(i+'/*.wav')

In [13]:
len(spk2utt_train)

972

In [14]:
count = 0
count_list = []
for i in spk2utt_train:
    count += len(spk2utt_train[i])
    count_list.append(len(spk2utt_train[i]))
print(count)

2332


In [15]:
with open(spk2utt_train_dict, 'wb') as handle:
    pickle.dump(spk2utt_train, handle)

with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

## Use multi-process to check read wav

In [16]:
manager = Manager()
spk2utt_train_len_dict_m = manager.list()
eer_m = manager.list()

In [17]:
spk2utt_train_splitlist = []
num_per_process = len(spk2utt_train) // 10
for i in range(9):
    spk2utt_train_splitlist.append([j for j in range(i*num_per_process, (i+1)*num_per_process)])
    spk2utt_train_len_dict_m.append({})
spk2utt_train_splitlist.append([j for j in range(9*num_per_process, len(spk2utt_train))])
spk2utt_train_len_dict_m.append({})

In [18]:
def read_wav_m(p, spk2utt_train, spk2utt_train_split, spk2utt_train_len_dict_m, eer_m):
    spk2utt_train_len_dict = {}
    for count, i in enumerate(spk2utt_train_split):
        for utt in spk2utt_train[i]:
            try:
                wav = librosa.core.load(utt, sr=16000)[0]
                wav_len = wav.shape[0]
                if (wav == 0.0).all():
                    eer_m.append([p, utt])
#                     print('a')
                else:
                    spk2utt_train_len_dict[utt] = wav_len
#                     print('b')
            except Exception:
                eer_m.append([p, utt])
                continue
        print(count)
    spk2utt_train_len_dict_m[p] = spk2utt_train_len_dict

In [19]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

processes = [Process(target = read_wav_m, args = (i, spk2utt_train, spk2utt_train_splitlist[i], spk2utt_train_len_dict_m, eer_m)) for i in range(10)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

0
0
0
0
0
1
0
2
1
0
1
0
1
1
1
2
3
3
2
4
0
1
2
2
4
1
2
0
1
3
3
5
5
2
3
4
6
1
2
6
4
3
5
3
2
5
7
4
2
6
8
7
9
7
5
8
6
10
8
6
4
3
9
4
7
5
5
4
8
6
5
11
9
10
7
6
3
9
3
10
11
8
10
4
7
12
11
12
4
9
10
13
12
6
13
8
14
11
5
7
13
9
5
11
6
6
12
10
8
15
11
9
13
7
12
14
13
8
10
14
15
9
15
14
1116

16
15
10
17
12
12
18
19
16
13
16
20
17
11
14
7
21
17
17
7
15
18
12
22
8
19
18
9
8
16
14
13
18
15
19
19
14
17
13
10
15
11
18
20
9
19
20
10
23
16
20
14
16
20
11
12
15
21
12
13
21
22
17
21
18
21
13
16
1423

17
22
19
24
22
18
24
25
20
15
23
19
25
23
20
24
21
26
14
16
21
26
15
22
22
16
27
23
27
25
24
26
24
17
23
24
17
28
25
18
18
25
27
26
25
29
26
19
30
28
27
20
22
19
27
23
21
28
29
29
28
30
30
22
26
31
23
24
31
31
32
25
29
24
20
26
27
32
28
28
33
21
34
30
25
29
22
35
26
23
33
31
30
27
29
17
28
24
18
36
31
19
30
27
32
29
37
20
28
32
31
30
33
38
31
39
29
21
40
32
25
2232

33
33
34
41
32
34
33
26
35
23
35
36
42
34
34
37
36
27
24
33
43
35
25
28
34
26
37
38
36
29
34
44
30
30
35
37
35
27
28
38
36
35
29
39
45
31
39
37

In [20]:
OPT_INDEX

'/workspace/DATASET/server9_ssd/STD_SRE_EGS'

In [21]:
spk2utt_train_len_dict = {}
for this_dict in spk2utt_train_len_dict_m:
    for count, i in enumerate(this_dict):
        spk2utt_train_len_dict[i] = this_dict[i]
#         print(count)
print('train_len_num', len(spk2utt_train_len_dict))

train_len_num 2332


In [22]:
err_list = list(eer_m)

In [23]:
total_len = 0
for i in range(10):
    total_len += len(spk2utt_train_len_dict_m[i])
print('total', total_len + len(err_list))
print('effect', total_len)
print('err', len(err_list))

total 2332
effect 2332
err 0


In [24]:
with open(spk2utt_train_len, 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict, handle)

# Make SRE18 indo split data

In [25]:
from preprocessing.Utils.preprocessing_logmel import logmel_IterableDataset

In [26]:
data_dir_dict = {}

data_dir_dict['spk2utt_train_dict'] = os.path.join(INFO_DIR, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(AUG_INFO_DIR, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(AUG_INFO_DIR, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(AUG_INFO_DIR, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(AUG_INFO_DIR, 'rir_dict')

In [27]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(INFO_DIR, 'spk2utt_train_len')
data_len_dict['music_len'] = os.path.join(AUG_INFO_DIR, 'music_len')
data_len_dict['noise_len'] = os.path.join(AUG_INFO_DIR, 'noise_len')
data_len_dict['babble_len'] = os.path.join(AUG_INFO_DIR, 'babble_len')

## Multi episode process

In [28]:
PROCESS = 200
RUNNING_P = 10
# RUNNING_P = 1

# MFCC_O = os.path.join(OPT_INDEX, 'MFCC')
LOGMELFB_O = os.path.join(OPT_INDEX, 'SRE18_INDO_FB')
# RAWWAV_O = os.path.join(OPT_INDEX, 'RW')
EER_LOG = os.path.join(OPT_INDEX, 'EER_LOG')
G_LOG = os.path.join(OPT_INDEX, 'G_LOG')

In [29]:
# if not os.path.exists(MFCC_O):
#     os.mkdir(MFCC_O)
# else:
#     shutil.rmtree(MFCC_O)
#     os.mkdir(MFCC_O)
    
if not os.path.exists(LOGMELFB_O):
    os.mkdir(LOGMELFB_O)
else:
    shutil.rmtree(LOGMELFB_O)
    os.mkdir(LOGMELFB_O)
    
# if not os.path.exists(RAWWAV_O):
#     os.mkdir(RAWWAV_O)
# else:
#     shutil.rmtree(RAWWAV_O)
#     os.mkdir(RAWWAV_O)

with open(EER_LOG, 'w') as f:
    pass

with open(G_LOG, 'w') as f:
    pass

In [30]:
config = {}

config['sr'] = 16000
config['repeats'] = 1
config['batch_size'] = 1

In [31]:
def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (rw, fb, mfcc, label) in enumerate(dataset):                
            file_name = str(i)+str('_')+str(count)
#             with open(os.path.join(RAWWAV_O, file_name), 'wb') as handle:
#                 pickle.dump((rw.astype(np.float16), label.astype(np.int16)), handle)
                
            with open(os.path.join(LOGMELFB_O, file_name), 'wb') as handle:
                pickle.dump((fb.astype(np.float16), label.astype(np.int16)), handle)
                
#             with open(os.path.join(MFCC_O, file_name), 'wb') as handle:
#                 pickle.dump((mfcc.astype(np.float16), label.astype(np.int16)), handle)
            
            with open(G_LOG, 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            
    except Exception:
        with open(EER_LOG, 'a') as f:
            traceback.print_exc(file=f)         
        raise Exception

dataset = logmel_IterableDataset(config, data_dir_dict, data_len_dict)
dataset.noise_data_preload()

p_list = []
for i in range(PROCESS // RUNNING_P):
    p_list.append([j for j in range(i*RUNNING_P, i*RUNNING_P+RUNNING_P)])
if (PROCESS % RUNNING_P) != 0:
    p_list.append([j for j in range((i+1)*RUNNING_P, PROCESS)])

print('episode', len(p_list))
print('last episode', p_list[-1])

start_time = time.time()
for ps in p_list:
    print(ps[0])
    processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in ps]
    [p.start() for p in processes]
    joined = [p.join() for p in processes]

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
episode 20
last episode [190, 191, 192, 193, 194, 195, 196, 197, 198, 199]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


# Make SRE18 indo split data index

In [32]:
# train_mfcc_dir = os.path.join(INFO_DIR, 'train_mfcc.csv')
train_fb_dir = os.path.join(INFO_DIR, 'train_indo_split_fb.csv')
# train_rw_dir = os.path.join(INFO_DIR, 'train_raw.csv')

In [36]:
LOGMELFB_O

'/workspace/DATASET/server9_ssd/STD_SRE_EGS/SRE18_INDO_FB'

In [33]:
# mfcc_len = glob.glob(MFCC_O+'/*')
fb_len = glob.glob(LOGMELFB_O+'/*')
# raw_len = glob.glob(RAWWAV_O+'/*')

In [34]:
# print('mfcc', len(mfcc_len))
print('fb', len(fb_len))
# print('raw', len(raw_len))

fb 194400


In [38]:
# count = 0
# with open(train_mfcc_dir, 'w') as f:
#     for path in mfcc_len:
#         assert os.path.isfile(path)
#         f.write(path+'\n')
#         count += 1
# print('mfcc', count)

count = 0
with open(train_fb_dir, 'w') as f:
    for path in fb_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('fb', count)

# count = 0
# with open(train_rw_dir, 'w') as f:
#     for path in raw_len:
#         assert os.path.isfile(path)
#         f.write(path+'\n')
#         count += 1
# print('raw', count)

fb 194400


In [37]:
972*200

194400

# Make SRE18 indo full data

In [68]:
from preprocessing.Utils.preprocessing_3type import ThreeTypes_IterableDataset

In [69]:
# MFCC_O = os.path.join(OPT_INDEX, 'SRE16_TEST_TRIALS_MFCC')
LOGMELFB_O = os.path.join(OPT_INDEX, 'SRE18_INDO_FULL_FB')
# RAWWAV_O = os.path.join(OPT_INDEX, 'SRE16_TEST_TRIALS_RAW')
EER_LOG = os.path.join(OPT_INDEX, 'EER_LOG')
G_LOG = os.path.join(OPT_INDEX, 'G_LOG')

In [70]:
# if not os.path.exists(MFCC_O):
#     os.mkdir(MFCC_O)
# else:
#     shutil.rmtree(MFCC_O)
    
if not os.path.exists(LOGMELFB_O):
    os.mkdir(LOGMELFB_O)
else:
    shutil.rmtree(LOGMELFB_O)
    
# if not os.path.exists(RAWWAV_O):
#     os.mkdir(RAWWAV_O)
# else:
#     shutil.rmtree(RAWWAV_O)

with open(EER_LOG, 'w') as f:
    pass

with open(G_LOG, 'w') as f:
    pass

In [71]:
config = {}

config['sr'] = 16000
config['repeats'] = 1
config['batch_size'] = 1

trial_dict_dir = os.path.join(INFO_DIR, 'spk2utt_train_dict')

In [ ]:
batched_feats_rawwave, batched_feats_LogMelFB, batched_feats_MFCC

In [72]:
def trial_data_preload(dataset, i, trial_dict_dir):
    with open(trial_dict_dir, 'rb') as handle:
        trial_dict = pickle.load(handle)
        
    start_time = time.time()
    for count, i in enumerate(trial_dict):
        for count_j, j in enumerate(trial_dict[i]):        
            rw, fb, mfcc = dataset.process_one_utt(j)
            label = i

            file_name = str(count)+'_'+str(count_j)
    #         with open(os.path.join(RAWWAV_O, file_name), 'wb') as handle:
    #             pickle.dump((rw.astype(np.float16), [label]), handle)
            with open(os.path.join(LOGMELFB_O, file_name), 'wb') as handle:
                pickle.dump((fb.astype(np.float16), [label]), handle)
    #         with open(os.path.join(MFCC_O, file_name), 'wb') as handle:
    #             pickle.dump((mfcc.astype(np.float16), [label]), handle)        
        if (count+1) % 500 == 0:
            print(count+1)

dataset = ThreeTypes_IterableDataset(config)

processes = [Process(target = trial_data_preload, args = (dataset, i, trial_dict_dir)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]

500


# Make SRE18 indo full data index

In [73]:
# trials_mfcc_dir = os.path.join(INFO_DIR, 'sre16_test_trials_mfcc.csv')
trials_fb_dir = os.path.join(INFO_DIR, 'train_indo_full_fb.csv')
# trials_rw_dir = os.path.join(INFO_DIR, 'sre16_test_trials_raw.csv')

In [74]:
# mfcc_len = glob.glob(MFCC_O+'/*')
fb_len = glob.glob(LOGMELFB_O+'/*')
# raw_len = glob.glob(RAWWAV_O+'/*')

In [75]:
# print('mfcc', len(mfcc_len))
print('fb', len(fb_len))
# print('raw', len(raw_len))

fb 2332


In [76]:
# count = 0
# with open(trials_mfcc_dir, 'w') as f:
#     for path in mfcc_len:
#         assert os.path.isfile(path)
#         f.write(path+'\n')
#         count += 1
# print('mfcc', count)

count = 0
with open(trials_fb_dir, 'w') as f:
    for path in fb_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('fb', count)

# count = 0
# with open(trials_rw_dir, 'w') as f:
#     for path in raw_len:
#         assert os.path.isfile(path)
#         f.write(path+'\n')
#         count += 1
# print('raw', count)

fb 2332
